In [ ]:
%load_ext lab_black

In [ ]:
import torch
import torch.nn as nn
from cnn import Img2Vec

from emoji_image_dataset import EmojiImageDataset

import pandas as pd

from embedding_analysis import EmbeddingAnalysis

### load data

In [ ]:
df = pd.read_csv("data/processed/emoji_descriptions.csv")[
    ["emoji_name", "emoji_char"]
].drop_duplicates()

In [ ]:
X_train = EmojiImageDataset("train")
df_train = X_train.df.reset_index(drop=True).merge(df)
df_train["old_index"] = df_train.index
df_train = df_train.drop_duplicates(subset="emoji_name").reset_index(drop=True)

In [ ]:
X_zero = EmojiImageDataset("zero")
df_zero = X_zero.df.reset_index(drop=True).merge(df)
df_zero["old_index"] = df_zero.index
df_zero = df_zero.drop_duplicates(subset="emoji_name").reset_index(drop=True)

### load model

In [ ]:
model = Img2Vec(200, "model/emoji_image_embedding/emimem.ckpt")
model.eval()

sim = nn.CosineSimilarity()

### create embeddings

In [ ]:
def create_embeddings(X, df):
    ls = []
    batch_size = 64
    for b in range(0, len(df), batch_size):
        batch = []
        for idx in df.old_index[b : b + batch_size]:
            batch.append(X[idx].unsqueeze(0))
        batch = torch.cat(batch, dim=0)
        o = model(batch)
        ls.append(o)

    embeddings = torch.concat(ls, dim=0)
    return embeddings

In [ ]:
%%time
embeddings_train = create_embeddings(X_train, df_train)
index_label_train = df_train.emoji_name.to_dict()
label_emoji = {k:v for k, v in zip(df_train.emoji_name, df_train.emoji_char)}
ea_train = EmbeddingAnalysis(embeddings_train, index_label_train, label_emoji)

In [ ]:
%%time 
embeddings_zero = create_embeddings(X_zero, df_zero)
index_label_zero = df_zero.emoji_name.to_dict()
label_emoji = {k:v for k, v in zip(df_zero.emoji_name, df_zero.emoji_char)}
ea_zero = EmbeddingAnalysis(embeddings_zero, index_label_zero, label_emoji)

In [ ]:
df_total = pd.concat([df_train, df_zero]).reset_index(drop=True)
index_label_total = df_total.emoji_name.to_dict()
label_emoji_total = {k: v for k, v in zip(df_total.emoji_name, df_total.emoji_char)}
embeddings_total = torch.cat([embeddings_train, embeddings_zero], dim=0)
ea_total = EmbeddingAnalysis(embeddings_total, index_label_total, label_emoji_total)

In [ ]:
df_total

In [ ]:
embeddings_total.shape

## similarity check

### training data

In [ ]:
df_train.sample(10)

In [ ]:
ea_train.most_similar("pensive_face")

In [ ]:
ea_train.most_similar("smiling_face_with_smiling_eyes")

In [ ]:
ea_train.most_similar("face_vomiting")

In [ ]:
ea_train.most_similar("see-no-evil_monkey")

In [ ]:
ea_train.most_similar("flag-_lebanon")

In [ ]:
ea_train.most_similar("eggplant")

### zero shot

In [ ]:
df_total.loc[df_total.dataset_type == "zero"].sample(10)

In [ ]:
ea_total.most_similar("sparkling_heart")

In [ ]:
ea_total.most_similar("weary_face")

In [ ]:
ea_total.most_similar("last_quarter_moon")

In [ ]:
ea_total.most_similar("flag-_bouvet_island")

In [ ]:
ea_total.most_similar("face_with_open_mouth")

In [ ]:
ea_total.most_similar("enraged_face")

In [ ]:
ea_total.most_similar("flag-_china")

In [ ]:
ea_total.most_similar("minibus")

In [ ]:
ea_total.most_similar("1st_place_medal")

In [ ]:
ea_total.most_similar("waffle")

In [ ]:
ea_total.most_similar("chart_increasing")

In [ ]:
ea_total.most_similar("sauropod")

In [ ]:
ea_total.most_similar("flexed_biceps")

In [ ]:
ea_total.most_similar("bacon")